In [2]:
import re
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import os
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from torchtext.vocab import Vocab, GloVe
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchtext.data import get_tokenizer


In [3]:
def data_cleaning(text):
    # 大小写
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace("#" , " ")
    text = text.replace("." , "")
    text = text.replace("%", "")



    text = re.sub('https?://[A-Za-z0-9./]+', '', text)
    text = re.sub('http?://[A-Za-z0-9./]+', '', text)
    text = re.sub('www.[A-Za-z0-9./]+', '', text)
    text = re.sub("\d+", "", text)
    text = re.sub("!+", "!", text)

#     text = '!'.join(unique_list(text.split('!')))
    text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

In [4]:
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist

a='I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== I AM AN LOSER == == I AM AN LOSER ==== '
a='='.join(unique_list(a.split('=')))

In [5]:
data_cleaning(" Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  Wikipedia tEh Free Enyclopedia!! ")

' Wikipedia tEh Free Enyclopedia! '

In [6]:
global_vectors = GloVe(name='twitter.27B', dim=200)

In [164]:
tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("?"), lower_case_backup=True)
s1 =  embeddings.tolist()
embeddings1 = global_vectors.get_vecs_by_tokens(tokenizer("fuck"), lower_case_backup=True)
s2 = embeddings1.tolist()

In [166]:
torch.sum(embeddings)

tensor(-9.1989)

In [7]:
data_folder_path = 'data/'
train = pd.read_csv(data_folder_path + 'train.csv')
test_data = pd.read_csv(data_folder_path + 'test.csv')
test_labels = pd.read_csv(data_folder_path + 'test_labels.csv')
test_data = pd.concat([test_data, test_labels], axis=1)

classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train[train[classes[1]] == 1].sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
103342,290a507a5d1a2fe6,"Hey sexist bitch, shut the fuck up\n\nHow dare...",1,1,1,0,1,0
37497,642701583b0cc86b,wikipedia loves cock \n\nwikipedia loves cock ...,1,1,1,0,0,0
133250,c8edbec909892107,Gogo blows man’s thick cock then gets on all f...,1,1,1,0,1,0
8865,1787e8af29b2563f,NJGW IS AN UNEMPLOYED MOTHER FUCKING BASTARD L...,1,1,1,0,1,0
132265,c3cea5cbd987b22a,Yo Wuhwuzdat u a bitch wit no life. Yeah u hea...,1,1,1,0,1,0
88801,ed948a2fda2f6272,"AND TO THE PARTY WHO FUCKS ME, FUCK U! I'LL BE...",1,1,1,1,1,0
32098,554a87fb67e51344,Fuck All Asyriac Nation \n\nQamishli belong to...,1,1,1,1,1,1
49353,83f81e6f784b74a6,it's unfortunate that your mother didn't quif ...,1,1,1,0,1,0
76543,ccf05c5d4b791836,"ur a faggot, fuck off and get a real job and s...",1,1,1,0,1,0
91858,f59198179d4305a5,Do you need a verifiable source for that state...,1,1,1,0,1,0


In [137]:
#seed = 42
#train, test_data = train_test_split(train, test_size=0.2, random_state=seed)


In [8]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [9]:
for cls in classes:
    rate = train[cls].sum() / len(train)
    rate = np.round(rate*100, 3)
    print(cls +' rate: ', rate, "%")

toxic rate:  9.584 %
severe_toxic rate:  1.0 %
obscene rate:  5.295 %
threat rate:  0.3 %
insult rate:  4.936 %
identity_hate rate:  0.88 %


In [10]:
for cls in classes:
    rate = test_data[cls].sum() / len(test_data)
    rate = np.round(rate*100, 3)
    print(cls +' rate: ', rate, "%")

toxic rate:  -54.253 %
severe_toxic rate:  -57.989 %
obscene rate:  -55.819 %
threat rate:  -58.091 %
insult rate:  -55.992 %
identity_hate rate:  -57.764 %


In [11]:
print(type(train))

<class 'pandas.core.frame.DataFrame'>


In [142]:
#l_train = []
l_train_drop =[]

for idx in range(0, len(train)):
    comment = train["comment_text"].iloc[idx]
    clean = data_cleaning(comment)
    token = tokenizer(clean)
    if len(token) == 0:
        l_train_drop.append(idx)
        continue
    train["comment_text"].iloc[idx] = global_vectors.get_vecs_by_tokens(token, lower_case_backup=True)


#    row['embeddings'] = embeddings

C:\Users\user\AppData\Local\Temp\ipykernel_25168\256433076.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["comment_text"].iloc[idx] = global_vectors.get_vecs_by_tokens(token, lower_case_backup=True)


In [143]:
train.drop(l_train_drop, axis = 0, inplace = True)

In [204]:
train['comment_text'][0].shape

torch.Size([51, 200])

In [205]:
train_new = train.reset_index(drop=True)

In [206]:
len(train['comment_text'])

159566

In [234]:
train_new['comment_text'][75496].shape
train_pad = train_new.copy()

In [235]:
max_len = 0
max_index = 0
max_id = 0
save = 0
for i in range(len(train_new['comment_text'])):
    if train_new['id'][i] == 'd1e3a54c4d490fe4':
        save = train_new['comment_text'][i]
    if len(train_new['comment_text'][i]) > max_len:
        max_len = len(train_new['comment_text'][i])
        max_index = i
        max_id = train_new['id'][i]
print(max_len)
print(max_index)
print(max_id)

310
134973
d1e56e18dadf4a24


In [ ]:
for i in range(len(train_pad['comment_text'])):
    count = 0
    for j in range(len(train_pad['comment_text'][i])):
        if torch.sum(train_pad['comment_text'][i][j]) == 0:
            count += 1
    #temp = torch.zeros([max_len+count-len(train_pad['comment_text'][i]),200])
    temp = torch.zeros([max_len,200])
    count_1 = 0
    for j in range(len(train_pad['comment_text'][i])):
        if torch.sum(train_pad['comment_text'][i][j]) != 0:
            #temp = torch.cat((temp, train_pad['comment_text'][i][j].reshape(1,-1)))
            temp[max_len+count-len(train_pad['comment_text'][i]) + count_1] = train_pad['comment_text'][i][j]
            count_1 += 1
    train_pad['comment_text'][i] = temp

C:\Users\user\AppData\Local\Temp\ipykernel_25168\3349311360.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pad['comment_text'][i] = temp


In [12]:
count = 0
for i in range(len(save)):
    if torch.sum(save[i]) == 0:
        count += 1
        
print(len(save))
temp = torch.zeros([max_len,200])
count_10 = 0
for i in range(len(save)):
    if torch.sum(save[i]) != 0:
        #save_non0 = torch.cat((save_non0, save[i].reshape(1,-1)))
        temp[max_len+count-len(save) + count_10] = save[i]
        count_10 += 1
save_non0 = temp
print(save_non0)

NameError: name 'save' is not defined

In [253]:
save_non0[282]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [195]:
save_non0 = save_non0[1:]

In [202]:
save_non0.shape

torch.Size([28, 200])

In [65]:
train_new.to_csv('train_after_tokenizer_clean_indexed.csv')

In [23]:
train.to_csv('train_after_tokenizer_clean.csv')

In [85]:
from torch.nn.utils.rnn import pad_sequence, pack_sequence

for idx in range(0, 5):
    comment = train["comment_text"][idx]
    print(comment)
    pad = pack_sequence(comment)
    print(pad)


tensor([[-0.1937, -0.1251, -0.5924,  ...,  0.7811, -0.1109, -0.2980],
        [ 0.2539,  0.3569,  0.2210,  ...,  0.1067, -0.1587,  0.1287],
        [ 0.4935,  0.3570,  0.6607,  ...,  0.1771, -0.5369, -0.2970],
        ...,
        [ 0.1430, -0.4178,  0.2529,  ..., -0.7805,  0.3792,  0.4934],
        [ 0.1588,  0.2819, -0.4703,  ...,  0.8744, -0.5519, -0.1927],
        [-0.0218, -0.0115, -0.4395,  ...,  0.4765,  0.2968,  0.6029]])
PackedSequence(data=tensor([-0.1937,  0.2539,  0.4935,  ...,  0.4934, -0.1927,  0.6029]), batch_sizes=tensor([51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
        51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 

In [ ]:
pad.data.shape

In [90]:
comment.shape

torch.Size([18, 200])

In [91]:
padded = pack_sequence(train["comment_text"])

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 631626054400 bytes. Error code 12 (Cannot allocate memory)